In [ ]:
import pandas as pd

# 读取Excel文件
excel_path = '/content/StateMinimumWage_Changes.xlsx'  # Colab中的Excel文件路径
df = pd.read_excel(excel_path)

# 筛选出没有1997年数据的州
states_without_1997 = df[~df['statename'].isin(df.loc[df['year'] == 1997, 'statename'])]['statename'].unique()
df_filtered = df[df['statename'].isin(states_without_1997)]

# 保存到CSV
output_csv_path = 'filtered_data_from_excel.csv'  # Colab中的文件名
df_filtered.to_csv(output_csv_path, index=False)

# 补充1990到1995年的数据
required_years = [1990, 1991, 1992, 1993, 1994, 1995, 1996]  # 包含1996
processed_df = pd.DataFrame()

for state in states_without_1997:
    state_data = df_filtered[df_filtered['statename'] == state].copy()
    for year in required_years:
        if year not in state_data['year'].values:
            new_row = {
                'statefips': state_data.iloc[0]['statefips'] if not state_data.empty else 'NaN',
                'statename': state,
                'year': year,
                'month': 1,  # 默认为1月
                'day': 1,    # 默认为1号
                'mw': 0,
                'changeinmw': 0
            }
            state_data = pd.concat([state_data, pd.DataFrame([new_row])], ignore_index=True)
    processed_df = pd.concat([processed_df, state_data], ignore_index=True)

# 排序并删除重复行
processed_df.sort_values(by=['statename', 'year'], inplace=True)
processed_df.drop_duplicates(subset=['statename', 'year'], keep='first', inplace=True)

# 计算changeinmw和changeinmw_ratio
processed_df['changeinmw'] = processed_df.groupby('statename')['mw'].diff().fillna(0).clip(lower=0)
processed_df['changeinmw_ratio'] = processed_df.groupby('statename')['changeinmw'].transform(lambda x: x / x.shift(1))
processed_df['changeinmw_ratio'] = processed_df['changeinmw_ratio'].replace([float('inf'), -float('inf')], 0).fillna(0)

# 保存到CSV
final_output_csv_with_ratio_path = 'final_processed_with_ratio_in_mw.csv'
processed_df.to_csv(final_output_csv_with_ratio_path, index=False)

print(f"Processed data with ratio in change in mw saved to {final_output_csv_with_ratio_path}")

import pandas as pd
from sklearn.linear_model import LassoCV

# 假设Arkansas州的数据已经按照你的要求处理好
arkansas_template = pd.DataFrame({
    'year': [1990, 1991, 1992, 1993, 1994, 1995],
    'changeinmw': [0, 0, 0.35, 0.15, 0.1, 0]  # 假设值，实际值应该是你计算出来的
})

# 读取最终的州际数据集
final_df = pd.read_csv('/mnt/data/final_processed_with_ratio_in_mw.csv')

# 填充缺失年份
filled_final_df = pd.DataFrame()
years_to_fill = range(1990, 1996)
for state in final_df['statename'].unique():
    state_data = final_df[final_df['statename'] == state]
    filled_years = state_data['year'].unique()
    for year in years_to_fill:
        if year not in filled_years:
            missing_row = {
                'statefips': state_data.iloc[0]['statefips'] if not state_data.empty else 'NaN',
                'statename': state,
                'year': year,
                'changeinmw': 0  # 填充0
            }
            state_data = pd.concat([state_data, pd.DataFrame([missing_row])], ignore_index=True)
    filled_final_df = pd.concat([filled_final_df, state_data], ignore_index=True)

# 排序并重置索引
filled_final_df.sort_values(by=['statename', 'year'], inplace=True)
filled_final_df.reset_index(drop=True, inplace=True)

# 进行双重Lasso回归以找到与Arkansas州最匹配的州
# 注意：下面的代码块是伪代码，因为双重Lasso需要特定的统计库实现，我在这里提供一个概念性框架
best_state = None
best_score = float('-inf')

for state in filled_final_df['statename'].unique():
    if state != 'Arkansas':
        state_data = filled_final_df[filled_final_df['statename'] == state]
        # 确保对比的年份一致
        state_data = state_data[state_data['year'].isin(arkansas_template['year'])]
        X = state_data[['year', 'changeinmw']].values
        y = arkansas_template['changeinmw'].values
        # 使用交叉验证的Lasso回归
        lasso = LassoCV(cv=5).fit(X, y)
        score = lasso.score(X, y)
        if score > best_score:
            best_score = score
            best_state = state

print(f"The state most similar to Arkansas based on changeinmw is: {best_state}")



/usr/local/lib/python3.10/dist-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Processed data with ratio in change in mw saved to final_processed_with_ratio_in_mw.csv


In [5]:


import pandas as pd
from sklearn.linear_model import LassoCV

# 假设Arkansas州的数据已经按照你的要求处理好
arkansas_template = pd.DataFrame({
    'year': [1990, 1991, 1992, 1993, 1994, 1995],
    'changeinmw': [0, 0, 0.35, 0.15, 0.1, 0]  # 假设值，实际值应该是你计算出来的
})

# 读取最终的州际数据集
final_df = pd.read_csv('/content/final_processed_with_ratio_in_mw.csv')

# 填充缺失年份
filled_final_df = pd.DataFrame()
years_to_fill = range(1990, 1996)
for state in final_df['statename'].unique():
    state_data = final_df[final_df['statename'] == state]
    filled_years = state_data['year'].unique()
    for year in years_to_fill:
        if year not in filled_years:
            missing_row = {
                'statefips': state_data.iloc[0]['statefips'] if not state_data.empty else 'NaN',
                'statename': state,
                'year': year,
                'changeinmw': 0  # 填充0
            }
            state_data = pd.concat([state_data, pd.DataFrame([missing_row])], ignore_index=True)
    filled_final_df = pd.concat([filled_final_df, state_data], ignore_index=True)

# 排序并重置索引
filled_final_df.sort_values(by=['statename', 'year'], inplace=True)
filled_final_df.reset_index(drop=True, inplace=True)

# 进行双重Lasso回归以找到与Arkansas州最匹配的州
# 注意：下面的代码块是伪代码，因为双重Lasso需要特定的统计库实现，我在这里提供一个概念性框架
best_state = None
best_score = float('-inf')

for state in filled_final_df['statename'].unique():
    if state != 'Arkansas':
        state_data = filled_final_df[filled_final_df['statename'] == state]
        # 确保对比的年份一致
        state_data = state_data[state_data['year'].isin(arkansas_template['year'])]
        X = state_data[['year', 'changeinmw']].values
        y = arkansas_template['changeinmw'].values
        # 使用交叉验证的Lasso回归
        lasso = LassoCV(cv=5).fit(X, y)
        score = lasso.score(X, y)
        if score > best_score:
            best_score = score
            best_state = state

print(f"The state most similar to Arkansas based on changeinmw is: {best_state}")



The state most similar to Arkansas based on changeinmw is: New York


In [4]:
import pandas as pd
import statsmodels.formula.api as smf
#DID Without machine learning enhance
# Load the data
df = pd.read_csv(r"/content/data_93-03.csv")

# Select data for Arkansas State
df_ar = df[df['statenum'] == 5].copy()

# Determine the first year of minimum wage change for Arkansas
first_policy_change_year = 1993

# Create a 'Post' variable for the years after the first policy change
df_ar['Post'] = (df_ar['year'] >= first_policy_change_year).astype(int)

# Since we're only interested in the effect after the first change,
# we need to create a 'Treat' variable that identifies the policy change
# For simplification, we'll consider a change in the minimum wage ('MW') as treatment
df_ar['Treat'] = (df_ar['year'] == first_policy_change_year) & (df_ar['MW'].diff() != 0).astype(int)

# Create the interaction term for the treatment effect after the policy change
df_ar['Post*Treat'] = df_ar['Post'] * df_ar['Treat']

# Define the regression formula, no state fixed effects are included since we are only analyzing Arkansas
# Year fixed effects are included to control for any time-specific effects
formula = 'emp_rate ~ Post*Treat + C(year)'

# Fit the DiD model
result_ar = smf.ols(formula, data=df_ar).fit()

# Print the regression results
print(result_ar.summary())

                            OLS Regression Results                            
Dep. Variable:               emp_rate   R-squared:                       0.778
Model:                            OLS   Adj. R-squared:                  0.777
Method:                 Least Squares   F-statistic:                     1636.
Date:                Wed, 24 Apr 2024   Prob (F-statistic):               0.00
Time:                        06:04:07   Log-Likelihood:                 12269.
No. Observations:                5148   AIC:                        -2.451e+04
Df Residuals:                    5136   BIC:                        -2.444e+04
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept              0.2200      0

In [2]:
import pandas as pd
import statsmodels.formula.api as smf

#Machine learning enhanced results
# Load the dataset
df = pd.read_csv(r"/content/data_93-03.csv")

# Identify Arkansas as the treatment group (5) and New York as the control group (36)
df['Treat'] = (df['statenum'] == 5).astype(int)
df['Control'] = (df['statenum'] == 36).astype(int)

# Since we're focusing on a multi-period DiD, we should create a post-treatment indicator
# for each year after the first policy change in Arkansas. This example assumes that the

# Find the years of policy changes in Arkansas
policy_change_years = df[df['statenum'] == 5]['year'].unique()
policy_change_years.sort()

# Remove years before the first policy change
policy_change_years = policy_change_years[policy_change_years >= 1993]

# Create a column for each post-treatment year
for year in policy_change_years:
  # Create a post-treatment indicator for this year
  df[f'Post_{year}'] = (df['year'] >= year).astype(int)

# We'll create an interaction term for the treatment group and each post-treatment year
for year in policy_change_years:
  df[f'Interaction_{year}'] = df[f'Post_{year}'] * df['Treat']

# Construct the formula for our DiD model, including the interaction terms and year fixed effects
interaction_terms = [f'Interaction_{year}' for year in policy_change_years]
formula = 'emp_rate ~ ' + ' + '.join(interaction_terms) + ' + C(year)'

# We restrict our dataset to the control and treatment groups
did_df = df[(df['Treat'] == 1) | (df['Control'] == 1)]

# Fit the DiD model using ordinary least squares (OLS)
model = smf.ols(formula, data=did_df).fit()

# Output the results
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:               emp_rate   R-squared:                       0.884
Model:                            OLS   Adj. R-squared:                  0.884
Method:                 Least Squares   F-statistic:                     3734.
Date:                Wed, 24 Apr 2024   Prob (F-statistic):               0.00
Time:                        05:58:32   Log-Likelihood:                 24268.
No. Observations:               10296   AIC:                        -4.849e+04
Df Residuals:                   10274   BIC:                        -4.833e+04
Df Model:                          21                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept            0.3753      0.001  